### Title: Land Cover Classification Using Landsat Imagery in Google Earth Engine

#### Introduction:
This notebook provides a detailed process of conducting land cover classification using Landsat imagery using the Google Earth Engine platform. 

The Part 1 of this notebook deals with relevant data collection processes that is required to conduct the analysis i.e., defining the helper functions required to retrieve Landsat data for a specified geographic location, date range, and cloud cover threshold, defining functions for creating interactive map for visualizing Landsat imagery, and collecting the training data required for building a classification model.

The Part 2 of this notebook utilizes the data collected in Part 1 and builds a classification model to identify landcover classes and creates a visualization to identify changes in land cover from one class to other. 

#### Data & Logistics:

1. Data Provided: The location data provided was a .kml file that represented the Big Coast Project area. This large polygon was then processed in QGIS to split the large polygon into small sized polygons representing the area.
2. Data Used: Out of the several split polygons, only one polygon was used to create this project. Apart from that, for creating a training sample we will require land classification data that can be used to train a Machine Learning model and assess its accuracy. In this regard, the training data was created using the "Land Cover of North America at 30 meters, 2020" created by USGS between dates "2019-01-01T00:00:00Z–2021-12-31T00:00:00" and accessed via earth engine API(https://developers.google.com/earth-engine/datasets/catalog/USGS_NLCD_RELEASES_2020_REL_NALCMS).
3. To process and estimate the biomass cover for the desired area, data from Global Ecosystem Dynamics Investigation (GEDI), was used (https://daac.ornl.gov/GEDI/guides/GEDI_L4B_Gridded_Biomass.html). This dataset provides estimates of mean aboveground biomass density (AGBD) based on observations from 2019-04-18 to 2021-08-04 for the entire globe. The dataset available was for the entire globe, barring most areas from our desired project locations. However, I did get some representative samples of data extracted that was used for estimating biomass. (This part is covered in details in notebook 2).   

In [11]:
# Import the required libraries

import ee
import geemap
import pandas as pd
import geopandas as gpd

In [12]:
# # Authenticate and initialize the earthengine library
# # Trigger the authentication flow.
# ee.Authenticate()

# # Initialize the library.
# ee.Initialize()

In [13]:
# Define the helper functions

def fetch_landsat_data(location: ee.Geometry, 
                       start_date: str, 
                       end_date: str, 
                       cloud_cover: int):
    """
    This function fetches the landsat data between a date range (start and end date)
    for a location and returns a landsat imagery containing bands between B1 and B7.
    :param location: desired geometric location (Point or Polygon)
    :param start_date: start date
    :param end_date: end date
    :param cloud_cover: percent of cloud cover
    :return:earth engine image collection
    """
    # Define the Landsat image collection
    landsat_image = ee.ImageCollection('LANDSAT/LC08/C02/T1_TOA'). \
        filterDate(ee.Date(start_date), ee.Date(end_date)). \
        filterBounds(location). \
        filterMetadata('CLOUD_COVER', 'less_than', cloud_cover).first(). \
        select('B[1-7]')

    return landsat_image


def create_map(location: ee.Geometry, image: ee.Image):
    """
    This function is in the context of the Google Earth Engine Python API,
    that takes in the location and image (landsat) and outputs an interactive 
    map that can be used for visualizing geospatial data and results.
    :param location: location to focus on the basemap
    :param image: landsat imagery to overlay on basemap
    :return: geemap Map object
    """
    Map = geemap.Map()
    Map.centerObject(location, 10)
    Map.addLayer(image, {
        'bands': ['B4', 'B3', 'B2'],
        'min': 0,
        'max': 0.4,
        'gamma': 1.4
    }, 'Landsat Composite')
    return Map